In [ ]:
# default_exp export

# Export

> Code that allows you to export a notebook (.ipynb) as a python script( .py) to a target folder.

ipynb2py will allow you to convert the notebook (.ipynb) where the function is executed to a python script. 

The conversion applies these rules: 

* The notebook will be automatically saved when the function is executed.
* Only code cells will be converted (not markdown cells).
* A header will be added to indicate the script has been automatically generated. It also indicates where the original ipynb is.
* Cells with a #hide flag won't be converted. Flag variants like # hide, #Hide, #HIDE, ... are also acceptable.
* Empty cells and unnecessary empty lines within cells will be removed.
* By default the script will be created with the same name and in the same folder of the original notebook. But you can pass a target folder and a different file name if you wish.
* If a script with the same name already exists, it will be overwriten.

In [ ]:
#export
"""
Code copied from the great nbdev library: https://github.com/fastai/nbdev/blob/master/nbdev/export.py
"""

def _mk_flag_re(body, n_params, comment):
    "Compiles a regex for finding nbdev flags"
    import re
    assert body!=True, 'magics no longer supported'
    prefix = r"\s*\#\s*"
    param_group = ""
    if n_params == -1: param_group = r"[ \t]+(.+)"
    if n_params == 1: param_group = r"[ \t]+(\S+)"
    if n_params == (0,1): param_group = r"(?:[ \t]+(\S+))?"
    return re.compile(rf"""
# {comment}:
^            # beginning of line (since re.MULTILINE is passed)
{prefix}
{body}
{param_group}
[ \t]*       # any number of spaces and/or tabs
$            # end of line (since re.MULTILINE is passed)
""", re.MULTILINE | re.VERBOSE)

_re_hide = _mk_flag_re("hide?", 0,
    "Matches any line with #hide without any module name")

In [ ]:
# export
def _get_unhidden_cells(cells):
    result = []
    for i,cell in enumerate(cells):
        if cell['cell_type'] == 'code': 
            if not _re_hide.findall(cell['source'].lower()) and cell['source'] != '': result.append(i)
    return result

def _save_nb(wait=2):
    """
    Save and checkpoints current jupyter notebook.
    """
    from IPython.core.display import Javascript, display
    import time
    display(Javascript('IPython.notebook.save_checkpoint();'))
    time.sleep(wait)
    
def _read_nb(fname):
    "Read the notebook in `fname`."
    from pathlib import Path
    import nbformat
    with open(Path(fname),'r', encoding='utf8') as f: return nbformat.reads(f.read(), as_version=4)

In [ ]:
# export
def get_nb_name():
    """
    Return the full path of the jupyter notebook.
    """
    import json
    import os
    import re
    import ipykernel
    import requests
    from urllib.parse import urljoin
    from notebook.notebookapp import list_running_servers
    kernel_id = re.search('kernel-(.*).json', ipykernel.connect.get_connection_file()).group(1)
    servers = list_running_servers()
    for ss in servers:
        response = requests.get(urljoin(ss['url'], 'api/sessions'), params={'token': ss.get('token', '')})
        for nn in json.loads(response.text):
            if nn['kernel']['id'] == kernel_id:
                relative_path = nn['notebook']['path']
                return os.path.join(ss['notebook_dir'], relative_path)

In [ ]:
# export
def nb2py(target='.', fname=None, verbose=True):
    """Converts the notebook where the function is run to a python script.
    
    Args:
        target : target directory where the script will be created. Defaults to current directory. 
        fname  : name of the script that will be created. Defaults to notebook name.
        verbose: prints out details of the export if True.        
    """
    
    import os
    from pathlib import Path
    try: import nbformat
    except ImportError: raise ImportError('You need to install nbformat to use nb2py!')
    
    # save nb
    _save_nb()
    
    # get py full script name
    if fname is not None: 
        if '.' in fname and fname.rsplit('.', 1)[1] == 'py': fname = Path(fname)
        else: fname = Path('.'.join([fname, 'py']))
    nb_path = get_nb_name()
    if nb_path is None or not os.path.isfile(nb_path):
        if fname is None:
            print("nb2py couldn't get the nb name as a default name; please pass an fname as an argument")
            return
        else: 
            nb_path = Path(os.getcwd())/fname
    nb_path = Path(nb_path)
    nb_name = nb_path.name
    if fname is None: pyname = Path('.'.join([nb_path.stem, 'py']))
    else: pyname = fname
    pyname = Path(target)/pyname
    
    # delete file if exists and create target folder if doesn't exist
    if os.path.exists(pyname): os.remove(pyname)
    pyname.parent.mkdir(parents=True, exist_ok=True)
    
    # Write script header
    with open(pyname, 'w') as f:
        f.write(f'# -*- coding: utf-8 -*-\n')
        f.write(f'"""{nb_name}\n\n')
        f.write(f'Automatically generated.\n\n')
        if nb_path is not None:
            f.write(f'Original file is located at:\n')
            f.write(f'    {nb_path}\n')
        f.write(f'"""')

    # identify convertible cells (excluding empty and those with hide flags)
    nb = _read_nb(nb_name)
    idxs = _get_unhidden_cells(nb['cells'])
    pnb = nbformat.from_dict(nb)
    pnb['cells'] = [pnb['cells'][i] for i in idxs]

    # clean up cells and write script
    sep = '\n'* 2
    for i,cell in enumerate(pnb['cells']):
        source_str = cell['source'].replace('\r', '')
        code_lines = source_str.split('\n')    
        if code_lines == ['']: continue
        while code_lines[0] == '': code_lines = code_lines[1:]
        while code_lines[-1] == '': code_lines = code_lines[:-1]
        cl = []
        for j in range(len(code_lines)): 
            if list(set(code_lines[j].split(" "))) == ['']:
                code_lines[j] = ''
            if i == 0 or code_lines[j-1] != '' or code_lines[j] != '': 
                cl.append(code_lines[j])
        code_lines = cl
        code = sep + '\n'.join(code_lines)
        with open(pyname, 'a', encoding='utf8') as f: f.write(code)
            
    # check script exists
    assert os.path.isfile(pyname), f"an error occurred during the export and {pyname} doesn't exist"
    if verbose: 
        print(f"{nb_name} converted to {pyname}")
    return str(pyname)

In [ ]:
# # hide
# import os
# from pathlib import Path
# import shutil
# pyname = nb2py(target='.', fname=None)
# if pyname is not None: 
#     pyname2 = Path('.'.join([get_nb_name().rsplit('.', 1)[0],'py'])).name
#     assert str(pyname) == str(pyname2)
#     assert os.path.isfile(pyname2)
#     os.remove(pyname2)
#     assert not os.path.isfile(pyname2)
    
# target = Path('test_export')
# pyname = nb2py(target=target, fname=None)
# if pyname is not None: 
#     pyname2 = target/Path('.'.join([get_nb_name().rsplit('.', 1)[0],'py'])).name
#     assert str(pyname) == str(pyname2)
#     assert os.path.isfile(pyname2)
#     shutil.rmtree(Path(pyname2).parent)
#     assert not os.path.isfile(pyname2)

In [ ]:
#hide
from tsai.imports import *
out = create_scripts(); beep(out)